
# Linear Models Solution — Price Prediction (Kaggle-ready)

This notebook builds a **high-performing linear baseline** fully compliant with the competition rules:
- Uses only **linear models** (Ridge and SGDRegressor with L1/ElasticNet penalties).
- Emphasizes **feature engineering**, regularization, and cross-validation.
- Produces a **`submission.csv`**.

> **Assumptions**: You have `train.csv` and `test.csv` with columns:  
`carat, depth, table, x, y, z, cut, color, clarity, price` (no `price` in `test.csv`).


In [49]:

import os
import sys
import math
import json
import random
import numpy as np
import pandas as pd

from typing import List, Tuple
from dataclasses import dataclass

from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge, SGDRegressor, RidgeCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV

import warnings
warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# Global seeds for reproducibility
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

pd.set_option('display.max_columns', 200)


In [50]:
# === Paths (edit if needed) ===
DATA_PATH = "XY_diamonds.csv"
SUBMISSION_PATH = 'submission.csv'

assert os.path.exists(DATA_PATH), f"Cannot find {DATA_PATH}. Please put train.csv in working directory."


In [51]:
raw = pd.read_csv(DATA_PATH)
has_price = raw["price"].notna()
print("Labeled rows:", has_price.sum(), "/", len(raw))

train = raw.loc[has_price].copy()
test  = raw.loc[~has_price].copy()

Labeled rows: 160000 / 200000


In [52]:

class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Add domain-inspired features and handle zero/NaN dimensions."""
    def __init__(self):
        # canonical ordinal maps (best is larger score)
        self.cut_map = {'F':1,'G':2,'V':3,'P':4,'I':5}  # Fair<Good<VeryGood<Premium<Ideal
        self.color_map = {'J':1,'I':2,'H':3,'G':4,'F':5,'E':6,'D':7}  # J worst -> D best
        self.clarity_map = {
            'I1':1, 'SI2':2, 'SI1':3, 'VS2':4, 'VS1':5, 'VVS2':6, 'VVS1':7, 'IF':8
        }  # ascending quality
        
        self.numeric_cols_ = None
        self.categorical_cols_ = None

    def fit(self, X, y=None):
        # remember column sets
        cols = list(X.columns)
        self.categorical_cols_ = [c for c in ['cut','color','clarity'] if c in cols]
        self.numeric_cols_ = [c for c in cols if c not in self.categorical_cols_ + ['price']]
        return self

    @staticmethod
    def _safe_div(a, b):
        out = np.divide(a, b, out=np.full_like(a, np.nan, dtype='float64'), where=(b!=0)&(~np.isnan(b)))
        return out

    def transform(self, X):
        X = X.copy()
        # Ensure numeric types for dimensions
        for c in ['carat','depth','table','x','y','z']:
            if c in X.columns:
                X[c] = pd.to_numeric(X[c], errors='coerce')

        # Zero-as-missing flags
        for c in ['x','y','z']:
            if c in X.columns:
                X[f'{c}_is_zero'] = (X[c] == 0).astype(int)
                X.loc[X[c] == 0, c] = np.nan

        # Basic geometric features
        X['volume'] = X[['x','y','z']].prod(axis=1, skipna=True)
        X['area_xy'] = X['x'] * X['y']
        X['size_sum'] = X[['x','y','z']].sum(axis=1, skipna=True)
        X['size_mean'] = X[['x','y','z']].mean(axis=1, skipna=True)
        X['size_max'] = X[['x','y','z']].max(axis=1, skipna=True)
        X['size_min'] = X[['x','y','z']].min(axis=1, skipna=True)

        # Ratios and derived metrics
        X['depth_pct'] = 100.0 * self._safe_div(X['z'], (X['x'] + X['y'])/2.0)
        X['xy_ratio']  = self._safe_div(X['x'], X['y'])
        X['z_to_xy']   = self._safe_div(X['z'], np.sqrt(X['x'] * X['y']))
        X['carat_per_vol'] = self._safe_div(X['carat'], X['volume'])
        # New ratios
        X['table_over_depth'] = self._safe_div(X['table'], X['depth'])
        X['z_over_carat'] = self._safe_div(X['z'], X['carat'])
        X['mean_xy'] = (X['x'] + X['y']) / 2.0
        X['deviation_depth_pct_from_ideal'] = (X['depth_pct'] - 61.5).abs()
        X['deviation_table_from_ideal'] = (X['table'] - 57.0).abs()
        X['elongation'] = self._safe_div(X[['x','y']].max(axis=1, skipna=True), X[['x','y']].min(axis=1, skipna=True))

        # Logs to handle skew
        for c in ['carat','x','y','z','volume','area_xy','size_sum','size_mean','deviation_depth_pct_from_ideal','deviation_table_from_ideal']:
            X[f'log1p_{c}'] = np.log1p(X[c])

        # Interactions (controlled)
        X['carat_x_depth']   = X['carat'] * X['depth']
        X['carat_x_table']   = X['carat'] * X['table']
        X['carat_x_depthpct']= X['carat'] * X['depth_pct']
        X['depth_x_table']   = X['depth'] * X['table']
        # New interactions
        X['carat_x_volume'] = X['carat'] * X['volume']
        X['carat_x_log_volume'] = X['carat'] * X['log1p_volume']
        X['depthpct_x_table'] = X['depth_pct'] * X['table']
        X['xy_ratio_x_carat'] = X['xy_ratio'] * X['carat']
        X['elongation_x_carat'] = X['elongation'] * X['carat']

        # Ordinal quality scores (plus keep raw categories for OHE)
        if 'cut' in X.columns:
            X['cut_score'] = X['cut'].map(self.cut_map).astype(float)
        if 'color' in X.columns:
            X['color_score'] = X['color'].map(self.color_map).astype(float)
        if 'clarity' in X.columns:
            X['clarity_score'] = X['clarity'].map(self.clarity_map).astype(float)

        return X


In [53]:

class Winsorizer(BaseEstimator, TransformerMixin):
    """Clip numeric features to [low_q, high_q] learned from training data only."""
    def __init__(self, numeric_cols: List[str], low=0.005, high=0.995):
        self.numeric_cols = numeric_cols
        self.low = low
        self.high = high
        self.bounds_ = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()
        for c in self.numeric_cols:
            if c in X.columns:
                lo, hi = X[c].quantile(self.low), X[c].quantile(self.high)
                if not np.isfinite(lo):
                    lo = X[c].min()
                if not np.isfinite(hi):
                    hi = X[c].max()
                self.bounds_[c] = (lo, hi)
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        for c, (lo, hi) in self.bounds_.items():
            if c in X.columns:
                X[c] = X[c].clip(lo, hi)
        return X


In [54]:

# Feature engineering
fe = FeatureEngineer()
train_fe = fe.fit_transform(train)
test_fe  = fe.transform(test)

# Identify columns for preprocessing
cat_cols = [c for c in ['cut','color','clarity'] if c in train_fe.columns]
num_cols = [c for c in train_fe.columns if c not in cat_cols + ['price']]

# Winsorize numeric columns (fit on train only; apply to both)
win = Winsorizer(numeric_cols=num_cols, low=0.005, high=0.995)
train_fe[num_cols] = win.fit_transform(train_fe[num_cols])
test_fe[num_cols]  = win.transform(test_fe[num_cols])

# Preprocessor: impute + scale numerics; impute + OHE categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_cols),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('ohe', OneHotEncoder(handle_unknown='ignore'))]), cat_cols),
    ]
)


In [55]:

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def get_models():
    # Ridge with a broad alpha search
    ridge = Ridge(alpha=1.0, random_state=RANDOM_STATE)
    ridge_alphas = np.logspace(-3, 4, 20)

    # SGDRegressor variants support sparse matrices (after OHE), good for L1/EN
    sgd_l1 = SGDRegressor(loss='squared_error', penalty='l1', random_state=RANDOM_STATE, max_iter=4000, early_stopping=True)
    sgd_en = SGDRegressor(loss='squared_error', penalty='elasticnet', random_state=RANDOM_STATE, max_iter=4000, early_stopping=True)

    grids = {
        'ridge': {'model': ridge, 'params': {'model__alpha': ridge_alphas}},
        'sgd_l1': {'model': sgd_l1, 'params': {'model__alpha': [1e-5, 3e-5, 1e-4, 3e-4, 1e-3]}},
        'sgd_en': {'model': sgd_en, 'params': {'model__alpha': [1e-5, 3e-5, 1e-4, 3e-4, 1e-3], 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}},
    }
    return grids

def cross_validated_oof_predictions(X, y, preprocessor, model, params, n_splits=5):
    # y is on price scale; we'll log-transform inside this function for training
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    oof = np.zeros(len(y), dtype=float)
    models = []
    mae_scores = []

    fold = 0
    for train_idx, val_idx in kf.split(X, y):
        fold += 1
        X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

        # pipeline: preprocess -> model, trained on log1p(price)
        pipe = Pipeline(steps=[
            ('prep', preprocessor),
            ('model', clone(model))
        ])

        # do inner grid search for this fold (optimize MAE)
        gs = GridSearchCV(pipe, params, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
        # log-transform target for training
        y_tr_log = np.log1p(y_tr.values)
        gs.fit(X_tr, y_tr_log)

        best = gs.best_estimator_
        pred_log = best.predict(X_va)
        pred = np.expm1(pred_log)
        oof[val_idx] = pred

        score = mae(y_va.values, pred)
        mae_scores.append(score)
        models.append(best)
        print(f"Fold {fold}: MAE={score:.4f} | Best params: {gs.best_params_}")

    print(f"CV MAE mean={np.mean(mae_scores):.4f}  std={np.std(mae_scores):.4f}")
    return oof, models, mae_scores

def blend_predictions(preds_list, maes):
    # inverse-variance-style weighting (weights ~ 1 / mae^2)
    inv_var = np.array([1.0/(m**2 + 1e-12) for m in maes])
    w = inv_var / inv_var.sum()
    blended = np.sum([w[i]*preds_list[i] for i in range(len(preds_list))], axis=0)
    return blended, w


In [56]:

y = train_fe['price']
X = train_fe.drop(columns=['price'])

model_grids = get_models()

oof_dict = {}
models_dict = {}
rmse_dict = {}

for name, spec in model_grids.items():
    print(f"\n=== {name.upper()} ===")
    oof, models, scores = cross_validated_oof_predictions(X, y, preprocessor, spec['model'], spec['params'], n_splits=5)
    oof_dict[name] = oof
    models_dict[name] = models
    rmse_dict[name] = np.mean(scores)

# Blend OOF predictions
preds_list = [oof_dict['ridge'], oof_dict['sgd_l1'], oof_dict['sgd_en']]
rmses = [rmse_dict['ridge'], rmse_dict['sgd_l1'], rmse_dict['sgd_en']]
blended_oof, weights = blend_predictions(preds_list, rmses)

print("\nOOF MAE by model:")
for k,v in rmse_dict.items():
    print(f"  {k}: {v:.4f}")
print(f"Blending weights (ridge, sgd_l1, sgd_en): {weights.round(4)}")

oof_mae = mae(y.values, blended_oof)
print(f"\nBlended OOF MAE: {oof_mae:.4f}")



=== RIDGE ===
Fold 1: MAE=670.9265 | Best params: {'model__alpha': np.float64(0.001)}
Fold 2: MAE=672.1233 | Best params: {'model__alpha': np.float64(0.001)}
Fold 3: MAE=671.0159 | Best params: {'model__alpha': np.float64(0.001)}
Fold 4: MAE=669.9550 | Best params: {'model__alpha': np.float64(0.002335721469090121)}
Fold 5: MAE=680.2884 | Best params: {'model__alpha': np.float64(0.001)}
CV MAE mean=672.8618  std=3.7763

=== SGD_L1 ===
Fold 1: MAE=684.3987 | Best params: {'model__alpha': 3e-05}
Fold 2: MAE=681.5689 | Best params: {'model__alpha': 1e-05}
Fold 3: MAE=686.3450 | Best params: {'model__alpha': 3e-05}
Fold 4: MAE=686.7716 | Best params: {'model__alpha': 1e-05}
Fold 5: MAE=693.9497 | Best params: {'model__alpha': 1e-05}
CV MAE mean=686.6068  std=4.1054

=== SGD_EN ===
Fold 1: MAE=684.6866 | Best params: {'model__alpha': 0.0003, 'model__l1_ratio': 0.1}
Fold 2: MAE=681.6892 | Best params: {'model__alpha': 3e-05, 'model__l1_ratio': 0.5}
Fold 3: MAE=686.2839 | Best params: {'model

In [57]:

# Refit best single model per type on full data (select the best fold estimator by its val MAE)
final_models = []

# For each model type, pick the fold-model with the lowest MAE and refit on FULL data
for name in ['ridge', 'sgd_l1', 'sgd_en']:
    specs = model_grids[name]
    # Find best fold index
    kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    fold_scores = []
    idx = 0
    for train_idx, val_idx in kf.split(X, y):
        y_va = y.iloc[val_idx]
        preds_va = oof_dict[name][val_idx]
        fold_scores.append(mae(y_va.values, preds_va))
    best_fold = int(np.argmin(fold_scores))

    best_estimator = models_dict[name][best_fold]
    # Refit on ALL data with the same hyperparameters
    params = best_estimator.get_params()
    model_refit = Pipeline(steps=[('prep', preprocessor), ('model', clone(best_estimator.named_steps['model']))])
    model_refit.named_steps['model'].set_params(**{k.replace('model__',''): v for k,v in best_estimator.get_params().items() if k.startswith('model__')})

    y_log = np.log1p(y.values)
    model_refit.fit(X, y_log)
    final_models.append((name, model_refit))

print("Refit complete for all base models.")


Refit complete for all base models.


In [61]:

# Predict with each final model
test_preds_each = []
for name, mdl in final_models:
    pred_log = mdl.predict(test_fe)
    pred = np.expm1(pred_log)
    test_preds_each.append(pred)

# Blend using the same weights computed from OOF
blended_test = np.sum([weights[i]*test_preds_each[i] for i in range(len(test_preds_each))], axis=0)

# Clean-up predictions
blended_test = np.maximum(blended_test, 0.0)  # no negative prices


In [62]:

# Determine ID column if present, else create 1..N ids
id_col = 'id' if 'id' in test.columns else None
if id_col:
    ids = test[id_col].astype(int).values
else:
    ids = np.arange(1, len(test) + 1, dtype=int)

# Build submission, clip/round per baseline reference
submission = pd.DataFrame({
    'id': ids,
    'price': blended_test
})
submission['price'] = submission['price'].clip(lower=10).round(2)

submission.to_csv(SUBMISSION_PATH, index=False)
print(f"Saved: {SUBMISSION_PATH}")
display(submission.head())


Saved: submission.csv


,id,price
0,1,841.48
1,2,15151.85
2,3,887.59
3,4,569.83
4,5,881.04


In [63]:

# Inspect top coefficients for the Ridge model (best fold refit on full data)
try:
    ridge_model = [m for n,m in final_models if n=='ridge'][0]
    # get feature names from preprocessor
    num_cols = [c for c in train_fe.columns if c not in ['price','cut','color','clarity']]
    cat_cols = [c for c in ['cut','color','clarity'] if c in train_fe.columns]
    ohe = ridge_model.named_steps['prep'].named_transformers_['cat']
    cat_features = list(ohe.get_feature_names_out(cat_cols)) if len(cat_cols)>0 else []
    feature_names = num_cols + cat_features

    coef = ridge_model.named_steps['model'].coef_
    # When StandardScaler + OneHotEncoder -> sparse design; coef_ is dense vector
    coef_df = pd.DataFrame({'feature': feature_names, 'coef': coef[:len(feature_names)]})
    coef_df['abs'] = coef_df['coef'].abs()
    coef_df = coef_df.sort_values('abs', ascending=False).head(25)
    display(coef_df[['feature','coef']])
except Exception as e:
    print("Coefficient inspection skipped:", e)


,feature,coef
21,mean_xy,-13.348361
30,log1p_area_xy,11.916922
4,y,11.417620
27,log1p_y,-10.447299
3,x,2.435368
32,log1p_size_mean,-1.882236
25,log1p_carat,1.478119
29,log1p_volume,-1.474877
26,log1p_x,-1.474384
12,size_mean,1.298084
